In [6]:
import numpy as np
import pandas as pd
import pickle

In [7]:
# Save dict 
def save_obj(obj, name ):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# Load dict
def load_obj(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

# load dict 
drug_target = load_obj('drug_target')

In [92]:
# Extract Drug-Target pairs with IC50 values
DTI = pd.DataFrame(columns=['drug', 'target', 'IC50', 'unit'])
for drug in tqdm(drug_target):
    for i in range(len(drug_target[drug]['target_chembl_id'])):
        if drug_target[drug]['type'][i]=='IC50':
            dict = {'drug':drug, 'target':drug_target[drug]['target_chembl_id'][i],
                    'IC50':drug_target[drug]['value'][i], 'unit':drug_target[drug]['units'][i]}
            DTI = DTI.append(dict, True)

# Save to file
DTI.to_csv('data/DTI.csv')

100%|██████████| 52498/52498 [55:51<00:00, 15.67it/s]


In [33]:
DTI.tail()

,Unnamed: 0,drug,target,IC50,unit
224179,224179,CHEMBL388956,CHEMBL331,1420.0,nM
224180,224180,CHEMBL388956,CHEMBL2185,2140.0,nM
224181,224181,CHEMBL388956,CHEMBL612545,35335.0,nM
224182,224182,CHEMBL388966,CHEMBL2431,10.0,nM
224183,224183,CHEMBL388966,CHEMBL2534,1.0,uM


In [31]:
# Summary Statistic
units, targets = [], []
for drug in drug_target:
    units.append(drug_target[drug]['type'])
    targets.append(drug_target[drug]['target_chembl_id'])
flatten_units = [j for sub in units for j in sub] 
flatten_targets = [j for sub in targets for j in sub] 

print('Measurements recorded: %s IC50, %s Kd and %s Ki'%(flatten_units.count('IC50'), flatten_units.count('Kd'), flatten_units.count('Ki')))
print('Total targets recorded: %s'%len(np.unique(np.array(flatten_targets))))
print('Targets with IC50 values: %s'%(len(DTI['target'].unique())))

Measurements recorded: 224184 IC50, 28763 Kd and 79203 Ki
Total targets recorded: 5724
Targets with IC50 values: 3146
